In [ ]:
# %load_ext autoreload
# %autoreload 2

In [2]:
from acemodel import AceModel, Children
from acemodel import default_mtd, corr_rand_mtd
from spew_data.spew import * 

In [3]:
import brfss
from brfss import bfs_data

In [4]:
# scbrfss = bfs_data('./brfss/SCBRFSS.csv')

In [5]:
data_prex = './spew_data/2010_ver1_45079_synth_'
cdl_data = cdn_data(data_prex)
len(cdl_data)

90314

In [6]:
def simulate_stack(mtd, step_num = 1000, display = False):    

    acemodel_list = [
        AceModel(cdl_data[
            (cdl_data[['_RACE_G1', '_INCOMG']] == [r,i]).all(axis = 1)
        ], r, i, step_method = mtd)
        for r in list(brfss.race_list.keys())[1:]
        for i in list(brfss.income_list.keys())[1:]
                    ]
    
    for i in range(step_num):
        [acemodel.step() for acemodel in acemodel_list]

    result = [res for res in [
            acemodel.datacollector.model_vars['Output'][-1]
            for acemodel in acemodel_list]
                if len(res) > 0
            ]
    
    result_df = pd.concat(result, axis = 0, 
                          ignore_index = True)
    
    if not display:
        return result_df
    
    result_bfs = bfs_data(result_df)
    
    for r in list(brfss.race_list.keys())[1:]:
        for i in list(brfss.income_list.keys())[1:]:
            print('RACE: ' + str(brfss.race_list[r]) + ' INCOME: ' + str(brfss.income_list[i]))
            display(result_bfs.get_corr_mat(r,i))
    return result_df

In [7]:
# scbrfss.get_corr_mat(1,1)

In [8]:
corr_model = AceModel(cdl_data[
    (cdl_data[['_RACE_G1', '_INCOMG']] == [1,1]).all(axis = 1)
], 1, 1, step_method = corr_rand_mtd)

In [ ]:
for i in range(1000):
    corr_model.step()
    
res = corr_model.datacollector.model_vars['Output'][-1]; res

In [ ]:
bfs_data(res).get_prop_mat(1,1)['pr']

In [10]:
bfs_data(res).get_corr_mat(1,1)

,depress,alcoabuse,drugabuse,prison,patdivorce,phyabuse1,phyabuse2,verbalabuse,sexabuse1,sexabuse2,sexabuse3,foodinsecure
depress,1.000000,0.054864,0.037640,0.046573,0.060552,0.015587,0.029731,0.018108,0.068593,0.057288,0.004550,0.026629
alcoabuse,0.054864,1.000000,0.048986,0.070871,-0.010529,0.040193,0.040725,0.047602,-0.004430,0.056109,0.046609,0.049310
drugabuse,0.037640,0.048986,1.000000,0.020453,0.017809,0.003304,0.015632,0.030325,-0.019482,0.032510,0.025366,0.026124
prison,0.046573,0.070871,0.020453,1.000000,0.065985,0.047579,-0.015416,-0.004377,0.030801,0.026814,-0.013160,0.025670
patdivorce,0.060552,-0.010529,0.017809,0.065985,1.000000,0.012407,0.052942,0.049760,0.029470,0.028646,-0.030883,0.099205
phyabuse1,0.015587,0.040193,0.003304,0.047579,0.012407,1.000000,0.066775,0.000556,0.060093,0.038164,0.006171,0.021352
phyabuse2,0.029731,0.040725,0.015632,-0.015416,0.052942,0.066775,1.000000,0.017937,0.039945,0.057321,0.055282,0.067925
verbalabuse,0.018108,0.047602,0.030325,-0.004377,0.049760,0.000556,0.017937,1.000000,0.078987,0.092195,0.068013,0.051824
sexabuse1,0.068593,-0.004430,-0.019482,0.030801,0.029470,0.060093,0.039945,0.078987,1.000000,0.118130,0.079691,0.068957
sexabuse2,0.057288,0.056109,0.032510,0.026814,0.028646,0.038164,0.057321,0.092195,0.118130,1.000000,0.180345,0.000917


In [1]:
default_result = simulate_stack(mtd = default_mtd, display = False)

NameError: name 'simulate_stack' is not defined

In [ ]:
corr_result = simulate_stack(mtd = corr_rand_mtd, display = False)